In [1]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import csv
from sklearn.utils import shuffle
import tensorflow as tf
import jovian
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import matplotlib

In [16]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import csv
from sklearn.utils import shuffle
import tensorflow as tf
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel , AutoModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm
import os 
import string 
import glob

In [53]:
device = torch.device('cpu') 
data_path   = '/Users/catalina.chirita/Desktop/Disertatie/'
train_path = data_path + "netflix_titles.csv"

pd_train = pd.read_csv(train_path, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
print(pd_train.head())

  show_id     type                  title         director  \
0      s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1      s2  TV Show          Blood & Water              NaN   
2      s3  TV Show              Ganglands  Julien Leclercq   
3      s4  TV Show  Jailbirds New Orleans              NaN   
4      s5  TV Show           Kota Factory              NaN   

                                                cast        country  \
0                                                NaN  United States   
1  Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   South Africa   
2  Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...            NaN   
3                                                NaN            NaN   
4  Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...          India   

           date_added  release_year rating   duration  \
0  September 25, 2021          2020  PG-13     90 min   
1  September 24, 2021          2021  TV-MA  2 Seasons   
2  September 24, 2021        

## Data Preparation and Cleaning

In [54]:
pd_train.drop(columns = ['title','director','cast','country','date_added','release_year','rating','duration','listed_in'], axis =1 , inplace = True)

In [55]:
print(pd_train.head())

  show_id     type                                        description
0      s1    Movie  As her father nears the end of his life, filmm...
1      s2  TV Show  After crossing paths at a party, a Cape Town t...
2      s3  TV Show  To protect his family from a powerful drug lor...
3      s4  TV Show  Feuds, flirtations and toilet talk go down amo...
4      s5  TV Show  In a city of coaching centers known to train I...


In [56]:
print('Number of training sentences: {:,}\n'.format(pd_train.shape[0]))

Number of training sentences: 8,807



In [57]:
pd_train = pd_train.replace('', 'null').replace(' ', 'null').dropna(subset=["description"])
print(pd_train.head())

  show_id     type                                        description
0      s1    Movie  As her father nears the end of his life, filmm...
1      s2  TV Show  After crossing paths at a party, a Cape Town t...
2      s3  TV Show  To protect his family from a powerful drug lor...
3      s4  TV Show  Feuds, flirtations and toilet talk go down amo...
4      s5  TV Show  In a city of coaching centers known to train I...


In [58]:
pd_train= pd_train.set_index('show_id')
print(pd_train.head())

            type                                        description
show_id                                                            
s1         Movie  As her father nears the end of his life, filmm...
s2       TV Show  After crossing paths at a party, a Cape Town t...
s3       TV Show  To protect his family from a powerful drug lor...
s4       TV Show  Feuds, flirtations and toilet talk go down amo...
s5       TV Show  In a city of coaching centers known to train I...


In [59]:
pd_train.groupby('type').describe()

description                                                            \
              count unique                                                top   
type                                                                            
Movie          6131   6105  Paranormal activity at a lush, abandoned prope...   
TV Show        2676   2672  This educational series for tiny tots features...   

              
        freq  
type          
Movie      4  
TV Show    2

In [60]:
df_movie = pd_train[pd_train['type'] == 'Movie']
df_movie.shape

(6131, 2)

In [61]:
df_tvshow = pd_train[pd_train['type'] == 'TV Show']
df_tvshow.shape

(2676, 2)

In [62]:
df_movie_downsampled = df_movie.sample(df_tvshow.shape[0])
df_balanced = pd.concat([df_tvshow,df_movie_downsampled])
print(df_balanced['type'].value_counts())

TV Show    2676
Movie      2676
Name: type, dtype: int64


In [63]:
df_balanced.sample(5)

,type,description
show_id,,
s1632,Movie,A wrong turn in the woods becomes a fight for ...
s4112,TV Show,"In the midnight hour, a lone DJ broadcasts the..."
s6077,Movie,"After losing his seventh job in a row, a TV pr..."
s2496,TV Show,Revisit the emotional bouts and memorable high...
s6222,Movie,"Half-dog, half-wolf Balto (voiced by Maurice L..."


In [64]:
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
df_balanced['description']= df_balanced['description'].apply(lambda x:remove_punctuation(x))
df_balanced['description']= df_balanced['description'].apply(lambda x: x.lower())

In [65]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords

df_balanced['description']= df_balanced['description'].apply(lambda x: word_tokenize(x))

final_stopwords_list = stopwords.words('english')
final_stopwords_list.append('«')
final_stopwords_list.append('»')
final_stopwords_list.append('’')
final_stopwords_list.append('-')
def remove_stopwords(text):
    output= [i for i in text if i not in final_stopwords_list]
    return output
def remove_digit(text):
    output= [c for c in text if not c.isdigit()]
    return output

df_balanced['description']= df_balanced['description'].apply(lambda x:remove_stopwords(x))
df_balanced['description']= df_balanced['description'].apply(lambda x:remove_digit(x))

for array in range(len(df_balanced['description'])):
    df_balanced['description'][array]=' '.join(df_balanced['description'][array]) 

In [66]:
df_balanced['description'][0]

'crossing paths party cape town teen sets prove whether privateschool swimming star sister abducted birth'

In [67]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = AutoModel.from_pretrained("bert-base-uncased")
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)
        return final_layer

In [68]:
from transformers import AutoTokenizer, AutoModel
import torch
# load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

labels = {'Movie':0,
          'TV Show':1
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['type']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['description']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y
    

In [69]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=4, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=4)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  

In [70]:
from sklearn.model_selection import train_test_split

np.random.seed(112)
df_train, df_val, df_test = np.split(df_balanced.sample(frac=1, random_state=42), 
                                     [int(.8*len(df_balanced)), int(.9*len(df_balanced))])

print(len(df_train),len(df_val), len(df_test))
                        

4281 535 536


In [ ]:
EPOCHS = 5
model = BertClassifier()
LR = 1e-6
              
train(model, df_train, df_val, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
 52%|█████▏    | 558/1071 [10:18:01<855:58:18, 6006.82s/it]

In [18]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

2022-05-13 16:02:05.588628: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

#Neural networks layers
l = tf.keras.layers.Dropout(0.1, name='droput')(outputs['pooled_output'])
l = tf.keras.layers.Dense(1,activation = 'sigmoid', name='output')(l)

#construct final model
model = tf.keras.Model(inputs = [text_input], outputs=[l])


In [25]:
outputs['pooled_output']

<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'keras_layer_1')>

In [21]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [22]:
print(pd.Series(X_train))

show_id
s5415    Akko enrolls at the Luna Nova Witchcraft Acade...
s7177    Award-winning documentarian Burns profiles thr...
s3242    It's the "Nailed It!" holiday special you've b...
s5566    With their mother in jail and bills piling up,...
s2896    Traumatized, violent and yearning for love, 9-...
                               ...                        
s6792    As Havana slowly revolves through the year, wi...
s3841    When everyone else mysteriously vanishes from ...
s1506    Love snackable, snap-worthy songs? Sing along ...
s4127    While investigating an actress’s supposed suic...
s6461    The virginal Tracey, who was raised in a stric...
Name: description, Length: 4014, dtype: object


In [23]:
X_train.shape

(4014,)

In [24]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs = 20)

Epoch 1/20
126/126 [==============================] - 489s 4s/step - loss: 0.7008 - accuracy: 0.5319 - precision: 0.5308 - recall: 0.5496
Epoch 2/20
126/126 [==============================] - 498s 4s/step - loss: 0.6804 - accuracy: 0.5655 - precision: 0.5663 - recall: 0.5595
Epoch 3/20
126/126 [==============================] - 481s 4s/step - loss: 0.6647 - accuracy: 0.5932 - precision: 0.5925 - recall: 0.5969
Epoch 4/20
126/126 [==============================] - 481s 4s/step - loss: 0.6539 - accuracy: 0.6101 - precision: 0.6074 - recall: 0.6228
Epoch 5/20
126/126 [==============================] - 483s 4s/step - loss: 0.6409 - accuracy: 0.6295 - precision: 0.6265 - recall: 0.6418
Epoch 6/20
126/126 [==============================] - 485s 4s/step - loss: 0.6396 - accuracy: 0.6348 - precision: 0.6331 - recall: 0.6413
Epoch 7/20
126/126 [==============================] - 479s 4s/step - loss: 0.6373 - accuracy: 0.6447 - precision: 0.6442 - recall: 0.6467
Epoch 8/20
126/126 [==============

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

In [ ]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, y_predicted)
cm 

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
print(classification_report(y_test, y_predicted))